In [50]:
import os
import json
import polars as pl

# Název složky, kde jsou uloženy JSON soubory s testy
directory_path = 'parsed_tests'

# Seznam pro ukládání všech otázek ze všech souborů
all_questions = []


# Procházení všech souborů v dané složce
print(f"Prohledávám složku: '{directory_path}'...")
for filename in os.listdir(directory_path):
    # Zpracováváme pouze soubory s příponou .json
    if filename.endswith('.json'):
        file_path = os.path.join(directory_path, filename)

        # Otevření a načtení JSON souboru
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)

                # Získání data testu
                test_date = data.get('datum_testu')
                questions = data.get('prehled_otazek', [])

                # Přidání data testu ke každé otázce a přidání do celkového seznamu
                for question in questions:
                    question['datum_testu'] = test_date
                    all_questions.append(question)

            except json.JSONDecodeError:
                print(f"Chyba při parsování souboru: {filename}")
            except Exception as e:
                print(f"Naskytla se neočekávaná chyba u souboru {filename}: {e}")


# Vytvoření Polars DataFrame ze seznamu všech otázek
if all_questions:
    df = pl.DataFrame(all_questions)

    # Převedení sloupce 'moznosti' z dict na struct pro lepší manipulaci v Polars
    # a datum_testu na datový typ Date
    df = df.with_columns(
        pl.col("moznosti").cast(pl.Struct([
            pl.Field("A", pl.Utf8),
            pl.Field("B", pl.Utf8),
            pl.Field("C", pl.Utf8)
        ])),
        pl.col("datum_testu").str.to_date("%d.%m.%Y", strict=False)
    )

    # Zobrazení prvních několika řádků a schématu DataFrame
    print("\nÚspěšně načteno a zpracováno:")
    print(f"Celkový počet otázek: {len(df)}")
    print("\nSchéma DataFrame:")
    print(df.schema)
else:
    print("\nNebyly nalezeny žádné otázky k načtení.")


print(f"Testy v rozsahu: {max(df['datum_testu'])} - {min(df['datum_testu'])}")
df

Prohledávám složku: 'parsed_tests'...
Naskytla se neočekávaná chyba u souboru test_966032.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966033.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966034.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966035.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966036.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966037.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966038.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966039.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966040.json: 'NoneType' object has no attribute 'get'
Naskytla se neočekávaná chyba u souboru test_966041.json: 'NoneType' ob

text_otazky,moznosti,spravna_odpoved,body,datum_testu
str,struct[3],str,i64,date
"""Osa zemská je myšlená přímka s…","{""zeměpisné"",""totožné"",""magnetické""}","""A""",3,2025-09-03
"""Hlavní příčinou atmosférické k…","{""rozdílná teplota vzduchu ve dne a v noci"",""dostatečný ohřev vzduchu o zemský povrch při instabilním zvrstvení"",""nestejnoměrné zahřívání různě barevného zemského povrchu""}","""B""",0,2025-09-03
"""Je-li dáván příkaz letadlu na …","{""pojíždění povoleno"",""opusťte přistávací plochu"",""vraťte se na místo odkud jste vyjel""}","""A""",1,2025-09-03
"""Odtržením proudnic rozumíme:""","{""skokový přechod od turbulentního proudění nad horní plochou křídla k absolutní"",""proudnice opustí profil, po tom co opíšou jeho tvar"",""proud vzduchu přestane sledovat tvar profilu""}","""C""",0,2025-09-03
"""Provádíme levou zatáčku o nákl…","{""skluzovou zatáčku"",""správnou zatáčku"",""výkluzovou zatáčku""}","""C""",0,2025-09-03
…,…,…,…,…
"""Údaj v metrech přepočítáte na …","{""(m 3) : 10"",""(m 3) + 10%"",""(m : 10) 3""}","""B""",0,2025-06-25
"""Nejspodnější vrstvu atmosféry …","{""mezosféra"",""troposféra"",""stratosféra""}","""B""",0,2025-06-25
"""Vyberte správné tvrzení týkají…","{""Postižený v bezvědomí následkem úrazu by měl být vždy uložen do polohy na zádech"",""Pokud postižený ve stávající poloze spolehlivě dýchá, ponecháme ho v této poloze."",""S postiženým v bezvědomí zásadně nehýbeme, abychom eliminovali riziko dalšího""}","""B""",0,2025-06-25


In [51]:
# Seskupení podle textu otázky a provedení agregace pro ostatní sloupce
df_grouped = df.group_by("text_otazky").agg(
    # Spočítáme, kolikrát se každá otázka vyskytla
    pl.len().alias("pocet_vyskytu"),

    # Pro sloupce, které by měly být pro danou otázku vždy stejné, vezmeme první výskyt
    pl.col("moznosti").first().alias("moznosti"),
    pl.col("spravna_odpoved").first().alias("spravna_odpoved"),

    # Vezmeme MAXIMÁLNÍ hodnotu bodů, abychom se "zbavili" nul, pokud existuje i nenulová varianta
    pl.col("body").max().alias("body"),

    # Zaznamenáme, zda pro danou otázku existovala i varianta s 0 body
    (pl.col("body") == 0).any().alias("obsahuje_variantu_s_0_body"),

    # Zjistíme datum, kdy se otázka objevila naposledy
    pl.col("datum_testu").max().alias("posledni_datum_testu")

).sort("pocet_vyskytu", descending=True) # Seřadíme výsledek podle četnosti

# Zobrazení výsledného unikátního a agregovaného DataFrame
print("Byl vytvořen nový DataFrame 'df_grouped' s unikátními otázkami.")
print(f"Původní počet záznamů: {len(df)}")
print(f"Nový počet unikátních otázek: {len(df_grouped)}")

df_grouped

Byl vytvořen nový DataFrame 'df_grouped' s unikátními otázkami.
Původní počet záznamů: 65972
Nový počet unikátních otázek: 938


text_otazky,pocet_vyskytu,moznosti,spravna_odpoved,body,obsahuje_variantu_s_0_body,posledni_datum_testu
str,u32,struct[3],str,i64,bool,date
"""Při laminárním proudění:""",732,"{""dochází k vzájemnému promíchávání proudnic, ale na konci sledovaného průřezu"",""nedochází k vzájemnému promíchávání proudnic"",""dochází k vzájemnému promíchávání proudnic""}","""B""",1,true,2025-09-05
"""Do jaké výšky lze počítat s „p…",664,"{""u ULLa do výšky přibližně 20 m"",""do výšky 1 m"",""asi do výšky jako je polovina rozpětí křídla""}","""C""",1,true,2025-09-05
"""Zeměkoule je:""",633,"{""síť souřadnicových čar"",""ideální koule"",""rotační elipsoid na pólech zploštělý""}","""C""",1,true,2025-09-05
"""Kritický bod je:""",515,"{""místo na trati kam až letadlo může doletět, aby se za současného stavu paliva mohlo"",""místo na trati plánované, ze kterého je stejná časová vzdálenost do místa startu i do"",""bod na plánované trati nejvíce vzdálený od VBT""}","""B""",1,true,2025-09-05
"""Těžiště letadla je:""",485,"{""působiště tíhové síly"",""působiště výsledné aerodynamické síly"",""působiště výsledné aerodynamické síly a tíhové síly""}","""A""",3,true,2025-09-05
…,…,…,…,…,…,…
"""Nebezpečnými aerodynamickými j…",1,"{""bez ohledu na konstrukci vrtulníku nízké otáčky, vírový prstenec rotoru a ocasní"",""nízké otáčky rotoru, vírový prstenec rotoru a ocasní vrtulky , podchvat, u vrtulníků"",""u vrtulníků schválených do provozu LAA nemohou nastat žádné nebezpečné""}","""B""",0,true,2025-09-05
"""Při nízkých otáčkách rotoru za…",1,"{""vlivem malé odstředivé síly , ale ještě dostatečného vztlaku před odtržením proudu"",""k odtržení proudu na listech, u dvoulistých rotorů s polotuhým zavěšením houpavého"",""vlivem malé odstředivé síly a ještě dostatečného vztlaku dojde ke katastrofálnímu""}","""C""",3,false,2025-09-05
"""Velkým nebezpečím při letu do …",1,"{""Nekontrolovatelné stoupání obalu"",""Deformace obalu (tzv. „vylití“) a tím ztráta Archimédovy síly následovaná prudkým"",""Deformace obalu a tím náhlé nekontrolovatelné otevření ZVV""}","""B""",0,true,2025-08-31


In [52]:
# Zkontrolujeme, zda DataFrame 'question_counts' existuje
if 'question_counts' in locals() and question_counts is not None and len(question_counts) > 0:
    # Importujeme potřebnou knihovnu pro vizualizaci
    import plotly.express as px

    # Vybereme prvních 10 nejčastějších otázek (DataFrame je již seřazen)
    top_10_questions = question_counts.head(50)

    # Vytvoření horizontálního sloupcového grafu
    # Tento typ grafu je ideální pro zobrazení dlouhých textových popisků
    fig = px.bar(
        top_10_questions,
        x="pocet_vyskytu",
        y="text_otazky",
        orientation='h', # Nastavení horizontální orientace
        title=f"Top 50 nejčastějších otázek v {len(os.listdir("parsed_tests"))} testech",
        labels={
            "pocet_vyskytu": "Počet výskytů v testech",
            "text_otazky": "Text otázky"
        },
        text="pocet_vyskytu" # Zobrazí hodnoty přímo na sloupcích
    )

    # Vylepšení vzhledu grafu
    # Seřadíme osu Y tak, aby otázka s nejvyšším počtem byla nahoře
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        height=600 # Zvětšíme výšku grafu, aby se texty vešly
    )

    # Zobrazení grafu
    fig.show()
    fig.to_image(format="png")
else:
    print("DataFrame 'question_counts' neexistuje nebo je prázdný. Spusťte nejprve buňku pro seskupení otázek.")

In [53]:
 # filter where points is 0
df_grouped.filter(pl.col("body") == 0).select([
    pl.col("text_otazky"),
    pl.col("spravna_odpoved"),
    pl.col("moznosti")
]).count()

text_otazky,spravna_odpoved,moznosti
u32,u32,u32
41,41,41
